# IT University of Copenhagen
## First year project - Project 2
### Investigating weather data and corona data in the Netherlands
#### Jacob Victor Enggaard Haahr, 
#### Christian Hugo Rasmussen 
#### Victoria Gonzalez
#### Lukas Rasocha

In [ ]:
#All the neccessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import json
from scipy.stats import pearsonr, spearmanr
import statsmodels.api as sm

In [ ]:
#Loading and filtering data

nl_covid = pd.read_csv('../data/Raw/corona/nl_corona.csv',sep='\t')
tests = pd.read_csv('../data/Raw/external/covid-19_tests.csv',sep=';')
weather = pd.read_csv('../data/Raw/weather/weather.csv',sep='\t')
with open('../data/Raw/metadata/nl_metadata.json') as f:
    metadata = json.load(f)
nl_weather = weather['iso3166-2'].str[:2] == 'NL'
nl_weather_df = weather[nl_weather]

reg_dict = {}
names = {}
for i in metadata['country_metadata']:
    reg_dict[int(i['covid_region_code'])] = [i['population']]
    names[int(i['covid_region_code'])] = i['iso3166-2_code']
    

reg_names = set(nl_covid['region_code'])
mask1 = nl_covid['confirmed_addition'].dropna()
mask_idk = nl_covid['confirmed_addition'].notnull()


for i in list(reg_names):
    mask = (nl_covid['region_code'] == i) & (nl_covid['confirmed_addition'].notnull())
    mask_covid=nl_covid[mask]
    reg_dict[i].append(sum(mask_covid['confirmed_addition']))
df1 = pd.DataFrame.from_dict(reg_dict)
df1 = df1.transpose()
final_df = df1.rename(names)

In [ ]:
#We have found an external dataset that used smaller subregions of the netherlands, we therefore renamed them to be included
#in the bigger regions that are defined by the gejson file

#making a list of all regionnames in tests which should be replaced
#with the nl_covid regions
gelder = ["Gelderland-Midden","Gelderland-Zuid","Noord- en Oost-Gelderland"]
over = ["IJsselland", "Twente"]
#flev = ["Flevoland"]
#Gron = ["Groningen"]
#Zee = ["Zeeland"]
Zhol = ["Haaglanden","Hollands-Midden","Rotterdam-Rijnmond","Zuid-Holland-Zuid"]
Nhol = ["Gooi en Vechtstreek", "Kennemerland", "Noord-Holland-Noord","Zaanstreek-Waterland","Amsterdam-Amstelland"]
#Utrecht = ["Utrecht"]
Lim = ["Limburg-Noord","Limburg-Zuid"]
Nbra = ["Brabant-Noord","Brabant-Zuidoost","Midden- en West-Brabant"]
Frie = ["Fryslân"]
#Dren = ["Drenthe"]
#Remove Onbekend

In [ ]:
#Replacing the regionnames with matching regions based on our data
tests["Security_region_name"].replace(gelder,"Gelderland", inplace=True)
tests["Security_region_name"].replace(over,"Overijssel", inplace=True)
tests["Security_region_name"].replace(Zhol,"Zuid-Holland", inplace=True)
tests["Security_region_name"].replace(Nhol,"Noord-Holland", inplace=True)
tests["Security_region_name"].replace(Lim,"Limburg", inplace=True)
tests["Security_region_name"].replace(Nbra,"Noord-Brabant", inplace=True)
tests["Security_region_name"].replace(Frie,"Friesland", inplace=True)

onbekendt_mask = (tests['Security_region_name'] != "Onbekend")

#removing unknown variable with data without a region
tests = tests[onbekendt_mask]

In [ ]:
#Changing the date from a string to a datetime class
tests['Date_of_statistics'] = pd.to_datetime(tests["Date_of_statistics"],format= "%d/%m/%Y")
#creating new column in the data frame for weekends
tests['weekend'] = (pd.to_datetime(tests['Date_of_statistics'],format = '%Y-%m-%d').dt.weekday >= 5).astype(int)

In [ ]:
'''Finding min, median, max and mean for each numerical value, for each region in the Netherlands.'''

num_columns = ['RelativeHumiditySurface','SolarRadiation','Surfacepressure','TemperatureAboveGround','Totalprecipitation','UVIndex','WindSpeed']
reg_dict = {}
for i in nl_weather_df['iso3166-2'].unique():
    reg_dict[i] = {}
    for j in num_columns:
        reg_dict[i][j] = {'min': min(nl_weather_df[nl_weather_df['iso3166-2'] == i][j]),'median':np.median(nl_weather_df[nl_weather_df['iso3166-2'] == i][j]),'max':max(nl_weather_df[nl_weather_df['iso3166-2'] == i][j]),'mean':np.mean(nl_weather_df[nl_weather_df['iso3166-2'] == i][j])}
#reg_dict['NL-GE']['RelativeHumiditySurface']

In [ ]:
#Calculate the mean per every date for all the numerical columns
date_dict = {}
for i in nl_weather_df['date'].unique():
    date_dict[i] = {}
    for j in num_columns:
        date_dict[i][j] = np.mean(nl_weather_df[nl_weather_df['date'] == i][j])
#date_dict

In [ ]:
#Plotting means of UVindices for different regions
means = [reg_dict[i]['UVIndex']['mean'] for i in reg_dict.keys()]

plt.xticks(rotation=90)
plt.bar(reg_dict.keys(),means);

In [ ]:
weather_by_day_nl = nl_weather_df.groupby(by = "date").mean()

weather_by_day_nl.loc[:, "UVIndex"].plot.line().legend(loc = "upper left")

In [ ]:
#Using the coordinates for different regions to plot them using folium
nl_map = folium.Map(location = [52.3,5.5], zoom_start = 7.4)
folium.GeoJson('../Data/Raw/shapefiles/nl.geojson', name = "geojson").add_to(nl_map)
folium.LayerControl().add_to(nl_map)

nl_map

In [ ]:
#Renaming dataframe's columns
final_df.columns = ['population', 'cases']
df = final_df.reset_index()
df.columns = ["region",'population', 'cases']

In [ ]:
#Create a new column in our data frame with the cases per capita
df["cases_pc"] = df["cases"] / df["population"]*100


nl_map = folium.Map(location = [52.3,5.5], zoom_start = 7.4)
folium.Choropleth(
    geo_data = '../Data/Raw/shapefiles/nl.geojson',
    name = "cases",
    data = df,
    columns = ['region', 'cases_pc'],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "number of cases %",
).add_to(nl_map)

nl_map

In [ ]:
#Cleaning covid data set by dropping columns that we didn't use and replacing names so we can merge it on that later on
mask = nl_covid['confirmed_addition'].notnull()
nl_covid_clean = nl_covid[mask]
nl_covid_clean = nl_covid_clean.drop(['deceased_addition','hospitalized_addition','deceased_cumulative','confirmed_cumulative','hospitalized_cumulative'], axis=1)
region_codes = set(nl_covid_clean['region_code'])

nl_covid_clean['region_code'].replace(list(names.keys()), list(names.values()), inplace=True)

In [ ]:
#Merge weather data with covid data
weather_corona = nl_covid_clean.merge(nl_weather_df, left_on=['date','region_code'],right_on=['date','iso3166-2'])
weather_corona.drop(['region_code'],axis=1) #remove duplicate column


In [ ]:
#Testing correlations for numerical variables in combined weather and covid dataframe - spearman, pearson, log

withoutzeros = weather_corona["confirmed_addition"].clip(lower=1) #enabling us to do log by changin 0's to 1's
sig_thresh = 0.001 / len(num_columns)*3

for var in num_columns:
    corr,p = pearsonr(weather_corona['confirmed_addition'],weather_corona[var])
    corr2,p2 = spearmanr(weather_corona['confirmed_addition'],weather_corona[var])
    corr3,p3 = pearsonr(np.log(withoutzeros),weather_corona[var])
    print('------------ '+var+' --------------')
    print("-----Pearson------")
    print("Correlation",corr)
    print("P value",p,p<sig_thresh)
    print("-----Spearman------")
    print("Correlation", corr2)
    print("P value", p2,p<sig_thresh)
    print("------LOG-------")
    print("Correlation", corr3)
    print("P value", p3,p<sig_thresh)
    print('\n')

In [ ]:
#dataframe containing only cases during spring lockdown
lockdown = (nl_covid['date'] > '2020-03-15') & (nl_covid['date'] < '2020-04-28')

yes = nl_covid[lockdown]
nl_covid_by_day =  yes.groupby(by='date').mean()
nl_covid_by_day.loc[:,'confirmed_addition'].plot.line(rot=90).legend(loc='upper left');

In [ ]:
#plotting covid and weather data by weeks, to better visualize data
plt.figure(0)
weather_corona['date'] = pd.to_datetime(weather_corona['date']) - pd.to_timedelta(7, unit='d')
df = weather_corona.groupby(['region_name', pd.Grouper(key='date', freq='W-MON')])['RelativeHumiditySurface'].sum().reset_index().sort_values('date')
df['RelativeHumiditySurface'] = df['RelativeHumiditySurface']/7
nl_covid_by_day =  df.groupby(by='date').mean()
nl_covid_by_day.loc[:,'RelativeHumiditySurface'].plot.line(rot=90).legend(loc='upper left')
plt.figure(1)
nl_covid['date'] = pd.to_datetime(nl_covid['date']) - pd.to_timedelta(7, unit='d')
df = nl_covid.groupby(['region_name', pd.Grouper(key='date', freq='W-MON')])['confirmed_addition'].sum().reset_index().sort_values('date')
df['confirmed_addition'] = df['confirmed_addition']/7
nl_covid_by_day =  df.groupby(by='date').mean()
nl_covid_by_day.loc[:,'confirmed_addition'].plot.line(rot=90).legend(loc='upper left')
plt.figure(2)
weather_corona['date'] = pd.to_datetime(weather_corona['date']) - pd.to_timedelta(7, unit='d')
df = weather_corona.groupby(['region_name', pd.Grouper(key='date', freq='W-MON')])['UVIndex'].sum().reset_index().sort_values('date')
df['UVIndex'] = df['UVIndex']/7
nl_covid_by_day =  df.groupby(by='date').mean()
nl_covid_by_day.loc[:,'UVIndex'].plot.line(rot=90).legend(loc='upper left')

In [ ]:
#Timeline of daily covid confirmed cases
#Colouring lines based on different events during the 2020 and partially 2021

first_lockdown = (nl_covid['date'] > '2020-03-15') & (nl_covid['date'] < '2020-04-28')
october_partial_lockdown = (nl_covid['date'] > '2020-10-14') & (nl_covid['date'] <= '2020-12-15')
hard_december_lockdown = (nl_covid['date'] > '2020-12-15') & (nl_covid['date'] < '2021-03-15')
plt.figure(figsize=(15,10))
nl_covid_by_day =  nl_covid.groupby(by='date').mean()
first_lockdown_by_day = nl_covid[first_lockdown].groupby('date').mean()
october_partial_lockdown_by_day  = nl_covid[october_partial_lockdown].groupby('date').mean()
hard_december_lockdown_by_day  = nl_covid[hard_december_lockdown].groupby('date').mean()

ax = nl_covid_by_day.loc[:,'confirmed_addition'].plot.line(color='k',rot=90,label='Confirmed addition') #covid addition daily
first_lockdown_by_day.loc[:,'confirmed_addition'].plot.line(color ='r',label='First lockdown',ax=ax) #first lockdown
october_partial_lockdown_by_day.loc[:,'confirmed_addition'].plot.line(color ='purple',label='Partial lockdown',ax=ax) #october partial lockdown
hard_december_lockdown_by_day.loc[:,'confirmed_addition'].plot.line(color ='darkgreen',label='Full lockdown',ax=ax) #october partial lockdown
plt.axvline('2020-12-19',label='Mutant strain ',color='slateblue')
plt.axvline('2021-01-09',label='Vaccination ',color='lawngreen')
plt.legend()
plt.show()

In [ ]:
#Calculate the average UVIndex per region
region_UVindex = {}

reg_names = set(weather_corona['region_code'])

for name in list(reg_names):
    mask = weather_corona['region_code'] == name
    region_UVindex[name] = [sum(weather_corona[mask]['UVIndex'])/len(weather_corona[mask])]

region_humidity_df = pd.DataFrame.from_dict(region_UVindex)
region_humidity_df = region_humidity_df.transpose().reset_index()
region_humidity_df.rename(columns = {'index':'region',0:'AVG UV'}, inplace = True)

In [ ]:
#Average UVIndex by region
nl_map = folium.Map(location = [52.3,5.5], zoom_start = 7.4)
folium.Choropleth(
    geo_data = '../Data/Raw/shapefiles/nl.geojson',
    name = "AVG UV",
    data = region_humidity_df,
    columns = ['region', 'AVG UV'],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "average UV",
).add_to(nl_map)

nl_map


In [ ]:
#Creating dummy variables for lockdown, holidays and weekends to see if any of them is correlated to the number of cases
Lockdown_True = (weather_corona['date'] > '2020-03-15') & (weather_corona['date'] < '2020-04-28') | (weather_corona['date'] > '2020-10-14') & (weather_corona['date'] <= '2020-12-15') | (weather_corona['date'] > '2020-12-15') & (weather_corona['date'] < '2021-03-15')
weather_corona['Lockdown'] = list(Lockdown_True)
weather_corona['Lockdown'].replace([True,False],[1,0],inplace = True)

In [ ]:
num_columns.append('holiday')
num_columns.append('Lockdown')
num_columns.append('weekend')
num_columns.append('no_school')
weather_corona['weekend'] = 0
weather_corona['weekend'] = (pd.to_datetime(weather_corona['date'],format = '%Y-%m-%d').dt.weekday >= 5).astype(int)
weather_corona['holiday'] = 0

no_school = (weather_corona['date'] > '2020-03-12') & (weather_corona['date']<'2020-06-08')
weather_corona['no_school'] = list(no_school)
weather_corona['no_school'].replace([True,False],[1,0],inplace=True)
weather_corona.loc[weather_corona['date'] == '2020-01-01','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-04-10','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-04-12','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-04-13','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-04-27','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-05-05','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-05-21','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-05-31','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-06-01','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-12-25','holiday'] = 1
weather_corona.loc[weather_corona['date'] == '2020-12-26','holiday'] = 1

In [ ]:
#Merging 2 dataframes to get the total number of cases and population in each of the regions, to be able to calculate cases per capita
df = final_df.reset_index()
df.columns = ["region",'population', 'cases']
weather_corona = weather_corona.merge(df, left_on=['region_code'],right_on=['region'])
weather_corona.drop('region',axis=1)

In [ ]:
#Multivariable analysis
weather_corona = sm.add_constant(weather_corona)
num_columns.append('const')
weather_corona['cases_per_c'] = weather_corona['confirmed_addition']/weather_corona['population']
est = sm.OLS(weather_corona['cases_per_c'],weather_corona[num_columns], hasconst = True).fit()
print(est.summary())

In [ ]:
#Multivariable analysis with our external dataset
tests['const'] = 1

est = sm.OLS(tests['Tested_positive'],tests[['Tested_with_result','weekend','const']], hasconst = True).fit()
print(est.summary())

sig_thresh = 0.000001 / 2

corr,p = pearsonr(tests['Tested_positive'],tests['Tested_with_result'])
corr2,p2 = spearmanr(tests['Tested_positive'],tests['Tested_with_result'])


print("-----Pearson------")
print("Correlation",corr)
print("P value",p,p<sig_thresh)
print("-----Spearman------")
print("Correlation", corr2)
print("P value", p2,p<sig_thresh)
print("------LOG-------")


In [ ]:
#Visualising the relationship between UV index and confirmed additions, trying to see if they are correlated

fig = plt.figure(figsize=(10,5))
axes = fig.add_axes([0,0,1,1])
axes.set_xlabel('Uv Index')
axes.set_ylabel('Confirmed addition')
axes.scatter(weather_corona['UVIndex'], weather_corona['confirmed_addition'],edgecolor='k',color='lawngreen');

In [ ]:
#Changing the data type of our dates in the external dataset
tests["Date_of_statistics"] = pd.to_datetime(tests["Date_of_statistics"],format= "%d/%m/%Y")

In [ ]:
#Needed to map the Region names (Drenthe etc.) to codes (NL-DR etc.) (to be able to visualize it on a map)

isos = sorted(list(set(nl_covid_clean['region_code'])))
names = sorted(list(set(nl_covid_clean['region_name'])))
isos_names = dict(zip(names,isos))

reg_dict = {}
for i in isos_names.keys():
    reg_dict[i] = []

for i in isos_names.keys():
    mask = (tests['Security_region_name'] == i) 
    mask2 = (weather_corona['region_name'] == i)
    mask_covid=tests[mask]
    mask_covid2 = weather_corona[mask2]
    reg_dict[i].append(isos_names[i])
    reg_dict[i].append(sum(mask_covid['Tested_with_result']))
    reg_dict[i].append(sum(mask_covid['Tested_positive']))
    reg_dict[i].append(list(set(mask_covid2['population']))[0])
    
df1 = pd.DataFrame.from_dict(reg_dict).transpose()
df1= df1.reset_index()
df1.rename(columns = {'index':'region',0:'region code', 1:'Total tests',2:'Tested positive',3:'Population'}, inplace = True)


df1['Per test'] = df1['Tested positive']/df1['Total tests']
df1['Tests per capita'] = df1['Total tests'] / df1['Population']

In [ ]:
#Percentage of populaton of people that got tested
#Maybe the less infected regions, are not what they seem to be

nl_map = folium.Map(location = [52.3,5.5], zoom_start = 7.4)
folium.Choropleth(
    geo_data = '../Data/Raw/shapefiles/nl.geojson',
    name = "Total tests and positive tests",
    data = df1,
    columns = ['region code', 'Tests per capita'],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Number of tests per capita",
).add_to(nl_map)

nl_map